In [1]:

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image



In [2]:

# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:

from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:


detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
import time
import cv2
import numpy as np
import ctypes
import _ctypes
import sys
from acquisitionKinect import AcquisitionKinect as ack
from frame import Frame as frame
import cv2
import pygame
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from skimage import img_as_ubyte
kinect = ack()
#kinect2 = ack()
frame = frame()
#frame2 = frame()
img_height = 424
img_width = 512
 
"""
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
filename="C:\\Users\\IVPL-D11\kinectREC\\output.avi"
codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
framerate=30
resolution=(640,480)
    
VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)
"""
def roi(image, vertices):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, vertices, 255)
    masked = cv2.bitwise_and(image, mask)
    return masked


with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    
    while True:
        kinect.get_frame(frame)
        kinect.get_color_frame()
      #  kinect.get_last_body_frame()
        image = kinect._frameRGB
        
        kinect.get_depth_frame()
        depthOriginal = kinect._frameDepth
        depthQuantized = kinect._frameDepthQuantized
        Skeleton_img = kinect._frameSkeleton
      #  depthOriginal = img_as_ubyte(depthOriginal)
     #   if kinect._bodies is not None:
     #       Skeleton_img = kinect._frameSkeleton
        mul_img = cv2.multiply(depthOriginal,np.array([12.0]))         # mul_img = img*alpha contrast [1.0-3.0]
        new_img = cv2.add((mul_img), np.array([120.0]))                 # new_img = img*alpha + beta brightness [0-100]
        new_img = cv2.GaussianBlur(new_img, (3,3), 0)
        #new_img = new_img.reshape(424,512,1) # <-- this makes it a rank-3 image
        #cvt_img = np.stack((0, 424, 512), axis =-1)
       #cvt_img = cv2.merge((new_img,new_img,new_img))
        #new_img = np.resize(new_img, (424, 512, 3))
      #  cvt_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
      #  cvt_img = cv2.cvtColor(cvt_img, cv2.COLOR_BGR2RGB)
       # new_img = cv2.equalizeHist(depthOriginal).astype('uint8')
        #OpenCv uses RGB image, kinect returns type RGBA, remove extra dim.
      #  cv2.resize(image, (512, 512))
        image_np = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB) 
        #vertices = np.array([[350, 200],[1780, 200],  [1780, 1020],[350, 1020],], np.int32)
        #image_np = roi(image_np, [vertices])
        image_np = image_np[ 200:1020, 350:1780]
        image_np = cv2.resize(image_np, (img_width, img_height))
        
      #  new_img = cv2.equalizeHist(new_img)
        #cv2.resize(image_np,dsize = (0,0),fx=0.5, fy = 0.5)
       # if not image is None:
        #    cv2.imshow("Output-Keypoints",image_np)


        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        #detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              new_img,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        box = np.squeeze(boxes)
        coordinates = vis_util.return_coordinates(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8,
                        min_score_thresh=0.80)

        
        if coordinates:
            ymin = coordinates[0]
            ymax = coordinates[1]
            xmin = coordinates[2]
            xmax = coordinates[3]
            print("left: ", ymin)
            print("right: ",ymax)
            print("top: ",xmin)
            print("bottom: ",xmax)
            cv2.rectangle(new_img,(xmin, ymin), (xmax, ymax), (255, 255,255),3)
    
        """
        left = boxline[0]
        print("left: ", left)
        right= boxline[1]
        print("right: ",right)
        top = boxline[2]
        print("top: ",top)
        bottom = boxline[3]
        print("bottom: ",bottom)
        """
        #cv2.rectangle(new_img, boxline[0] * img_height,boxline[1] * img_width, boxline[2] * img_height, boxline[3] * img_width)
        
        #print([category_index.get(i) for i in classes[0]])
        #print(scores)
        #VideoFileOutput.write(image_np)
        cv2.imshow('live_detection',image_np)
        cv2.imshow('depth', new_img)
        if kinect._bodies is not None:
            cv2.imshow('body', Skeleton_img)
        
       # cv2.imshow('skeleton', Skeleton_img)
      #  game.run();
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
            kinect.close()
            cv2.destroyAllWindows()
            cap.release()

left:  246
right:  340
top:  425
bottom:  510
left:  247
right:  340
top:  431
bottom:  510
left:  14
right:  415
top:  195
bottom:  390
left:  247
right:  341
top:  427
bottom:  509
left:  6
right:  417
top:  232
bottom:  486
left:  244
right:  342
top:  422
bottom:  509
left:  245
right:  344
top:  422
bottom:  509
left:  245
right:  346
top:  423
bottom:  510
left:  247
right:  345
top:  424
bottom:  510
left:  245
right:  342
top:  423
bottom:  509
left:  244
right:  344
top:  423
bottom:  509
left:  246
right:  342
top:  431
bottom:  510
left:  247
right:  341
top:  427
bottom:  509
left:  246
right:  341
top:  430
bottom:  511
left:  246
right:  341
top:  427
bottom:  510
left:  247
right:  340
top:  430
bottom:  510
left:  247
right:  340
top:  430
bottom:  511
left:  246
right:  340
top:  427
bottom:  509
left:  245
right:  343
top:  428
bottom:  510
left:  243
right:  342
top:  437
bottom:  511
left:  244
right:  342
top:  428
bottom:  510
left:  244
right:  342
top:  421
bott

left:  118
right:  424
top:  12
bottom:  269
left:  119
right:  424
top:  7
bottom:  271
left:  113
right:  424
top:  4
bottom:  267
left:  130
right:  424
top:  4
bottom:  284
left:  144
right:  424
top:  0
bottom:  259
left:  130
right:  423
top:  6
bottom:  269
left:  104
right:  424
top:  8
bottom:  276
left:  108
right:  423
top:  9
bottom:  276
left:  77
right:  424
top:  0
bottom:  279
left:  128
right:  421
top:  0
bottom:  218
left:  125
right:  420
top:  0
bottom:  215
left:  121
right:  421
top:  0
bottom:  226
left:  117
right:  423
top:  0
bottom:  227
left:  122
right:  421
top:  1
bottom:  223
left:  123
right:  421
top:  1
bottom:  217
left:  125
right:  421
top:  1
bottom:  220
left:  125
right:  421
top:  1
bottom:  218
left:  125
right:  421
top:  0
bottom:  217
left:  123
right:  420
top:  0
bottom:  219
left:  123
right:  420
top:  1
bottom:  220
left:  125
right:  423
top:  0
bottom:  247
left:  135
right:  424
top:  0
bottom:  243
left:  20
right:  419
top:  341


left:  169
right:  403
top:  46
bottom:  219
left:  168
right:  385
top:  49
bottom:  212
left:  166
right:  375
top:  47
bottom:  209
left:  167
right:  372
top:  49
bottom:  208
left:  168
right:  377
top:  49
bottom:  207
left:  167
right:  379
top:  51
bottom:  206
left:  167
right:  374
top:  52
bottom:  207
left:  168
right:  373
top:  51
bottom:  206
left:  169
right:  382
top:  49
bottom:  201
left:  170
right:  378
top:  45
bottom:  195
left:  168
right:  381
top:  42
bottom:  194
left:  169
right:  381
top:  40
bottom:  196
left:  171
right:  384
top:  38
bottom:  195
left:  169
right:  371
top:  43
bottom:  198
left:  169
right:  369
top:  44
bottom:  198
left:  171
right:  378
top:  42
bottom:  197
left:  169
right:  371
top:  45
bottom:  199
left:  168
right:  371
top:  46
bottom:  199
left:  169
right:  370
top:  46
bottom:  198
left:  169
right:  374
top:  43
bottom:  198
left:  170
right:  375
top:  45
bottom:  200
left:  172
right:  398
top:  39
bottom:  213
left:  169

left:  1
right:  422
top:  2
bottom:  486
left:  0
right:  423
top:  16
bottom:  470
left:  3
right:  420
top:  28
bottom:  472
left:  2
right:  422
top:  63
bottom:  509
left:  153
right:  421
top:  129
bottom:  448
left:  129
right:  423
top:  99
bottom:  439
left:  116
right:  424
top:  78
bottom:  431
left:  101
right:  424
top:  32
bottom:  432
left:  72
right:  424
top:  25
bottom:  411
left:  57
right:  423
top:  34
bottom:  399
left:  42
right:  421
top:  70
bottom:  423
left:  42
right:  422
top:  76
bottom:  456
left:  68
right:  423
top:  100
bottom:  508
left:  86
right:  424
top:  68
bottom:  510
left:  89
right:  423
top:  95
bottom:  508
left:  90
right:  422
top:  118
bottom:  509
left:  91
right:  424
top:  98
bottom:  512
left:  73
right:  424
top:  85
bottom:  512
left:  48
right:  424
top:  106
bottom:  512
left:  36
right:  421
top:  80
bottom:  511
left:  17
right:  421
top:  71
bottom:  510
left:  4
right:  420
top:  72
bottom:  504
left:  20
right:  421
top:  31